### Server Setup

Creating a stream of data & sending it to the clients.

<br>
<br>

In [1]:
# to specify delay between two consecutive records in stream
DELAY = 0.1

In [2]:
# Importing required modules

# data streaming
import asyncio
import websockets
from time import sleep

# data processing
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
# Read csv files of merged data

# # original data
# data = pd.read_csv("../data/IoT_Garage_Door.csv", low_memory=False)

# test data for presentation purpose
data = pd.read_csv("../data/IoT_Linux_Disk.csv", low_memory=False)

In [4]:
# dropping NaN or NA values
data = data.dropna()

# encoding string type variables to numeric type (required for model training)
encoder=LabelEncoder()
data['type']=encoder.fit_transform(data['type'])
data['PID']=encoder.fit_transform(data['PID'])
data['RDDSK']=encoder.fit_transform(data['RDDSK'])
data['WRDSK']=encoder.fit_transform(data['WRDSK'])
data['WCANCL']=encoder.fit_transform(data['WCANCL'])
data['DSK']=encoder.fit_transform(data['DSK'])
data['CMD']=encoder.fit_transform(data['CMD'])

In [5]:
data.head()

,ts,PID,RDDSK,WRDSK,WCANCL,DSK,CMD,label,type
0,1554218915,1318,279,1125,5,23,369,0,4
1,1554218920,642,805,1492,65,5,456,0,4
2,1554218925,951,595,1546,117,5,655,0,4
3,1554218930,955,861,333,0,3,710,0,4
4,1554218935,1119,1070,134,0,3,752,0,4


In [6]:
async def server(websocket):
    """
    Server function to stream data row-wise in a comma separated string format.
    """

    # convert a row to space(" ") separated string format
    temp = data.to_string(header=False,index=False,
                  index_names=False).split('\n')

    # now convert above row to comma(",") separated string format
    rows = [','.join(ele.split()) for ele in temp]
    
    c=0    # variable to keep track of number of records sent

    # while 'c' is less than & equal to total number of records in dataset
    # this loop is responsible for creating data stream
    while(len(data)>=c):
        # send stringified row to the client
        await websocket.send(rows[c])

        # wait of acknowledgement
        ok = await websocket.recv()
        if(bool(ok)):
            c+=1    # if successfully received then increment 'c' by 1
        
        # add artificial delay to simulate a remote server client setup having communication delays due to network traffic
        sleep(DELAY)

async def main():
    async with websockets.serve(server, "localhost", 8765, ping_interval=None):
        await asyncio.Future()  # run forever

await main()

connection handler failed
Traceback (most recent call last):
  File "C:\Users\spars\AppData\Roaming\Python\Python310\site-packages\websockets\legacy\server.py", line 231, in handler
    await self.ws_handler(self)
  File "C:\Users\spars\AppData\Local\Temp\ipykernel_8300\1634059650.py", line 22, in server
    ok = await websocket.recv()
  File "C:\Users\spars\AppData\Roaming\Python\Python310\site-packages\websockets\legacy\protocol.py", line 552, in recv
    await self.ensure_open()
  File "C:\Users\spars\AppData\Roaming\Python\Python310\site-packages\websockets\legacy\protocol.py", line 929, in ensure_open
    raise self.connection_closed_exc()
websockets.exceptions.ConnectionClosedOK: received 1000 (OK); then sent 1000 (OK)


<br>
<br>
<center><b>End of File</b></center>